<a href="https://colab.research.google.com/github/Osterneck/SEC_10K_analyzer_112825/blob/main/Copy_of_SEC_10K_parser_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



 ## SEC 10-K Financial Statement Extractor - V.8 (XBRL VERSION) (c) ai70000,Ltd.

**Presenter:** Alex Osterneck, CLA, MSCS  
**Organization:** ai70000, Ltd.  
**Date:** November 28th, 2025  
  

---

My jn-notebook code to access SEC Form 10-K filings: parsed, extracted, and printed to your local machine screen...no more hunting through pages of documents...instant access to: balance-sheet, income-statement, cash-flow statements.  V.9 to include conversion-code for visualizations and analytics.

In [ ]:
# pip install requests pandas beautifulsoup4 lxml

In [ ]:
#!/usr/bin/env python3
"""
SEC 10-K Financial Statement Extractor - V.8 (XBRL VERSION)
[note: V.9 to include visualizations and analytics]
Uses SEC's structured XBRL data for surgical accuracy to extract:
Summaries of: Balance-Sheet, Income-Statement, Cash-Flow Statement
Works universally across ALL U.S. public companies
(c) ai70000, Ltd., // Alex Osterneck, CLA, MSCS, MSIT
"""

import requests
import pandas as pd
import json
from time import sleep
from typing import Dict, Optional, Tuple
import sys
import warnings
warnings.filterwarnings('ignore')

class SEC10KFetcher:

    def __init__(self, user_agent: str):
        self.base_url = "https://www.sec.gov"
        self.headers = {
            'User-Agent': user_agent,
            'Accept-Encoding': 'gzip, deflate'
        }

    def get_cik_from_ticker(self, ticker: str) -> Optional[str]:
        """Get CIK for ticker"""
        ticker = ticker.upper().strip()
        try:
            url = "https://www.sec.gov/files/company_tickers.json"
            response = requests.get(url, headers=self.headers, timeout=10)
            response.raise_for_status()

            companies = response.json()
            for company in companies.values():
                if company['ticker'] == ticker:
                    cik = str(company['cik_str']).zfill(10)
                    print(f"✓ Found CIK: {cik}")
                    return cik

            print(f"✗ Ticker '{ticker}' not found")
            return None
        except Exception as e:
            print(f"✗ Error fetching CIK: {e}")
            return None

    def get_latest_10k_info(self, cik: str) -> Optional[Tuple[str, str]]:
        """Get latest 10-K filing accession number"""
        try:
            url = f"https://data.sec.gov/submissions/CIK{cik}.json"
            response = requests.get(url, headers=self.headers, timeout=10)
            response.raise_for_status()

            data = response.json()
            filings = data.get('filings', {}).get('recent', {})

            forms = filings.get('form', [])
            accession_numbers = filings.get('accessionNumber', [])
            filing_dates = filings.get('filingDate', [])

            for i, form in enumerate(forms):
                if form == '10-K':
                    accession = accession_numbers[i].replace('-', '')
                    print(f"✓ Found 10-K filed on: {filing_dates[i]}")
                    print(f"  Accession: {accession_numbers[i]}")
                    return (cik.lstrip('0'), accession)

            print("✗ No 10-K filing found")
            return None
        except Exception as e:
            print(f"✗ Error fetching 10-K info: {e}")
            return None

    def get_company_facts(self, cik: str) -> Optional[dict]:
        """Get company facts (XBRL data) from SEC"""
        try:
            # Use the 10-digit CIK with leading zeros
            url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
            print(f"\n  Fetching XBRL data from SEC API...")

            response = requests.get(url, headers=self.headers, timeout=15)
            response.raise_for_status()

            data = response.json()
            print(f"✓ Downloaded XBRL data ({len(str(data)):,} bytes)")
            return data

        except Exception as e:
            print(f"✗ Error fetching XBRL data: {e}")
            return None

    def extract_financials_from_xbrl(self, facts_data: dict, cik_no_zeros: str, accession: str) -> Dict[str, pd.DataFrame]:
        """Extract financial statements from XBRL company facts"""

        print("\n" + "="*80)
        print("EXTRACTING FINANCIAL STATEMENTS FROM XBRL DATA")
        print("="*80 + "\n")

        results = {
            'balance_sheet': None,
            'income_statement': None,
            'cash_flow': None
        }

        # Get the facts organized by taxonomy
        us_gaap = facts_data.get('facts', {}).get('us-gaap', {})
        dei = facts_data.get('facts', {}).get('dei', {})

        if not us_gaap:
            print("✗ No US-GAAP data found")
            return results

        print(f"Found {len(us_gaap)} US-GAAP concepts\n")

        # Find the most recent 10-K filing in the data
        target_form = '10-K'

        # Extract Balance Sheet
        print("Extracting Balance Sheet...")
        balance_sheet_concepts = {
            'Assets': 'Assets',
            'AssetsCurrent': 'Current Assets',
            'CashAndCashEquivalentsAtCarryingValue': 'Cash and Cash Equivalents',
            'MarketableSecurities': 'Marketable Securities',
            'AccountsReceivableNetCurrent': 'Accounts Receivable, Net',
            'InventoryNet': 'Inventory',
            'PropertyPlantAndEquipmentNet': 'Property, Plant and Equipment, Net',
            'Goodwill': 'Goodwill',
            'IntangibleAssetsNetExcludingGoodwill': 'Intangible Assets, Net',
            'Liabilities': 'Total Liabilities',
            'LiabilitiesCurrent': 'Current Liabilities',
            'AccountsPayableCurrent': 'Accounts Payable',
            'LongTermDebt': 'Long-term Debt',
            'StockholdersEquity': 'Stockholders\' Equity',
            'CommonStockValue': 'Common Stock',
            'RetainedEarningsAccumulatedDeficit': 'Retained Earnings',
            'LiabilitiesAndStockholdersEquity': 'Total Liabilities and Equity',
        }

        balance_sheet = self._extract_statement(us_gaap, balance_sheet_concepts, target_form, accession)
        if balance_sheet is not None:
            results['balance_sheet'] = balance_sheet
            print(f"✓ Balance Sheet: {len(balance_sheet)} line items\n")
        else:
            print("✗ Balance Sheet extraction failed\n")

        # Extract Income Statement
        print("Extracting Income Statement...")
        income_statement_concepts = {
            'Revenues': 'Revenue',
            'RevenueFromContractWithCustomerExcludingAssessedTax': 'Revenue',
            'CostOfRevenue': 'Cost of Revenue',
            'GrossProfit': 'Gross Profit',
            'OperatingExpenses': 'Operating Expenses',
            'ResearchAndDevelopmentExpense': 'Research and Development',
            'SellingGeneralAndAdministrativeExpense': 'Selling, General and Administrative',
            'OperatingIncomeLoss': 'Operating Income',
            'InterestExpense': 'Interest Expense',
            'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest': 'Income Before Taxes',
            'IncomeTaxExpenseBenefit': 'Income Tax Expense',
            'NetIncomeLoss': 'Net Income',
            'EarningsPerShareBasic': 'EPS - Basic',
            'EarningsPerShareDiluted': 'EPS - Diluted',
        }

        income_statement = self._extract_statement(us_gaap, income_statement_concepts, target_form, accession)
        if income_statement is not None:
            results['income_statement'] = income_statement
            print(f"✓ Income Statement: {len(income_statement)} line items\n")
        else:
            print("✗ Income Statement extraction failed\n")

        # Extract Cash Flow Statement
        print("Extracting Cash Flow Statement...")
        cash_flow_concepts = {
            'NetCashProvidedByUsedInOperatingActivities': 'Net Cash from Operating Activities',
            'NetIncomeLoss': 'Net Income',
            'DepreciationDepletionAndAmortization': 'Depreciation and Amortization',
            'PaymentsToAcquirePropertyPlantAndEquipment': 'Capital Expenditures',
            'NetCashProvidedByUsedInInvestingActivities': 'Net Cash from Investing Activities',
            'NetCashProvidedByUsedInFinancingActivities': 'Net Cash from Financing Activities',
            'RepaymentsOfLongTermDebt': 'Repayments of Long-term Debt',
            'PaymentsOfDividends': 'Dividends Paid',
            'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect': 'Net Change in Cash',
        }

        cash_flow = self._extract_statement(us_gaap, cash_flow_concepts, target_form, accession)
        if cash_flow is not None:
            results['cash_flow'] = cash_flow
            print(f"✓ Cash Flow Statement: {len(cash_flow)} line items\n")
        else:
            print("✗ Cash Flow Statement extraction failed\n")

        return results

    def _extract_statement(self, us_gaap: dict, concept_map: dict, target_form: str, accession: str) -> Optional[pd.DataFrame]:
        """Extract a financial statement from XBRL data"""

        data_rows = []

        for xbrl_concept, display_name in concept_map.items():
            if xbrl_concept not in us_gaap:
                continue

            concept_data = us_gaap[xbrl_concept]
            units = concept_data.get('units', {})

            # Try USD first, then other units
            unit_data = units.get('USD', units.get('USD/shares', units.get('shares', [])))

            if not unit_data:
                continue

            # Find entries from the target 10-K filing
            annual_data = {}

            for entry in unit_data:
                # Filter for 10-K filings (annual data)
                if entry.get('form') != target_form:
                    continue

                # Get fiscal year end date
                end_date = entry.get('end')
                if not end_date:
                    continue

                # Get the value
                value = entry.get('val')
                if value is None:
                    continue

                # Store by year (use fiscal year end)
                fiscal_year = end_date[:4]

                # Prefer entries from the specific accession we're looking for
                if entry.get('accn') == accession:
                    annual_data[fiscal_year] = value
                elif fiscal_year not in annual_data:
                    # Use this as fallback if we don't have data for this year yet
                    annual_data[fiscal_year] = value

            if annual_data:
                data_rows.append({
                    'Line Item': display_name,
                    **{year: val for year, val in sorted(annual_data.items(), reverse=True)}
                })

        if not data_rows:
            return None

        # Create DataFrame
        df = pd.DataFrame(data_rows)
        df = df.set_index('Line Item')

        # Sort columns by year (descending)
        year_columns = sorted([col for col in df.columns], reverse=True)
        df = df[year_columns]

        return df

    def fetch_10k_financials(self, ticker: str) -> Dict[str, pd.DataFrame]:
        """Main method to fetch and extract 10-K financials"""

        print("\n" + "="*80)
        print(f"FETCHING 10-K FOR: {ticker.upper()}")
        print("="*80 + "\n")

        # Get CIK
        cik = self.get_cik_from_ticker(ticker)
        if not cik:
            return {}

        sleep(0.1)

        # Get filing info
        filing_info = self.get_latest_10k_info(cik)
        if not filing_info:
            return {}

        cik_no_zeros, accession = filing_info
        sleep(0.1)

        # Get XBRL company facts
        facts_data = self.get_company_facts(cik)
        if not facts_data:
            return {}

        # Extract financial statements
        financials = self.extract_financials_from_xbrl(facts_data, cik_no_zeros, accession)

        # Summary
        success_count = sum(1 for df in financials.values() if df is not None)
        print("="*80)
        print(f"EXTRACTION COMPLETE: {success_count}/3 statements extracted")
        print("="*80 + "\n")

        return financials


def main():
    """Main execution function"""
    print("\n" + "="*80)
    print("SEC 10-K FINANCIAL STATEMENT EXTRACTOR - V.8 (XBRL)")
    print("="*80 + "\n")

    # Get user inputs
    email = input("Enter your email (required by SEC): ").strip()
    if not email or '@' not in email:
        print("\n✗ Invalid email address")
        sys.exit(1)

    ticker = input("Enter ticker symbol (e.g., AAPL, MSFT, TSLA): ").strip().upper()
    if not ticker:
        print("\n✗ Invalid ticker")
        sys.exit(1)

    # Create fetcher and extract financials
    fetcher = SEC10KFetcher(user_agent=email)
    financials = fetcher.fetch_10k_financials(ticker)

    # Display results
    if any(df is not None for df in financials.values()):
        print("\n" + "#"*80)
        print(f"{ticker} - FINANCIAL STATEMENTS".center(80))
        print("#"*80 + "\n")

        for key, name in [('balance_sheet', 'BALANCE SHEET'),
                         ('income_statement', 'INCOME STATEMENT'),
                         ('cash_flow', 'CASH FLOW STATEMENT')]:

            df = financials.get(key)
            if df is not None:
                print("="*80)
                print(name.center(80))
                print("="*80)

                # Clean display - replace NaN with empty string
                df_display = df.copy()
                df_display = df_display.fillna('')

                print(df_display.to_string())
                print("\n")

        # Save option
        save = input("Save to Excel file? (y/n): ").strip().lower()
        if save == 'y':
            filename = f"{ticker}_10K_Financials.xlsx"
            try:
                with pd.ExcelWriter(filename, engine='openpyxl') as writer:
                    for key, name in [('balance_sheet', 'Balance Sheet'),
                                     ('income_statement', 'Income Statement'),
                                     ('cash_flow', 'Cash Flow')]:
                        df = financials.get(key)
                        if df is not None:
                            # Fill NaN for Excel export
                            df_export = df.fillna('')
                            df_export.to_excel(writer, sheet_name=name)

                print(f"\n✓ File saved: {filename}\n")
            except Exception as e:
                print(f"\n✗ Error saving file: {e}\n")
    else:
        print("\n⚠ WARNING: No financial statements were extracted")
        print("This could mean:")
        print("  - The company hasn't filed XBRL data")
        print("  - The ticker doesn't have a recent 10-K")
        print("  - Network/SEC access issues\n")


if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\nProcess interrupted by user\n")
    except Exception as e:
        print(f"\n✗ FATAL ERROR: {e}")
        import traceback
        traceback.print_exc()


SEC 10-K FINANCIAL STATEMENT EXTRACTOR - V.8 (XBRL)

Enter your email (required by SEC): datascience68@gmail.com
Enter ticker symbol (e.g., AAPL, MSFT, TSLA): CAT

FETCHING 10-K FOR: CAT

✓ Found CIK: 0000018230
✓ Found 10-K filed on: 2025-02-14
  Accession: 0000018230-25-000008

  Fetching XBRL data from SEC API...
✓ Downloaded XBRL data (6,246,249 bytes)

EXTRACTING FINANCIAL STATEMENTS FROM XBRL DATA

Found 696 US-GAAP concepts

Extracting Balance Sheet...
✓ Balance Sheet: 13 line items

Extracting Income Statement...
✓ Income Statement: 11 line items

Extracting Cash Flow Statement...
✓ Cash Flow Statement: 8 line items

EXTRACTION COMPLETE: 3/3 statements extracted


################################################################################
                           CAT - FINANCIAL STATEMENTS                           
################################################################################

                                 BALANCE SHEET                            

---


---

**Presenter Contact:**
- Alex Osterneck, CLA, MSCS
- Email: aosterneck@ai70000.pro
- Specialization: DataScience, AI Engineering, Software Engineering for Accountancy. These specializations are transforming the accountancy profession by automating routine tasks and enabling a shift to more strategic, advisory roles. These fields offer specialized skills which allow professionals to manage large datasets, develop custom solutions, and provide deeper financial insights.



In [ ]:
from google.colab import files
ticker = 'CAT'  # enter CASE-Sensistive ticker-symbol to download your xlsx. you're welcome.
files.download(f'{ticker}_10K_Financials.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>